In [1]:
# Datenverarbeitung
import pandas as pd

# Visualisierung
import altair as alt

# Deaktivierung der Warnungen von Altair
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Ermöglicht die Darstellung von umfrangreichen Datensätzen
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [3]:
# Globale Variablen
domain = ['precious_metals', 'industrial_metals', 'grains', 'livestocks', 'energy', 'softs']
range_ = ['gold', 'grey', 'green', 'red', 'darkblue', 'lightblue']

In [4]:
df = pd.read_csv("data/commodity_df.csv")

In [5]:
df.head()

,date,year,month,category,mean_category,natural_gas,wti_crude,brent_crude,low_sulphur_gas_oil,uls_diesel,...,corn,soybean_oil,soybean_meal,wheat,hrw_wheat,live_cattle,lean_hogs,sugar,coffee,cotton
0,2000-01-03,2000,1,energy,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000-01-04,2000,1,energy,66.6792,2.176,25.55,24.39,213.50,67.78,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000-01-05,2000,1,energy,66.0716,2.168,24.91,23.73,213.00,66.55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000-01-06,2000,1,energy,65.6252,2.196,24.78,23.62,211.25,66.28,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000-01-07,2000,1,energy,63.8966,2.173,24.22,23.09,205.25,64.75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Diagramm 2
# Dieses Diagramm zeigt die durchschnittlichen Rohstoffpreise eines Jahres
# Zur Auswahl des Jahres wurde ein Select-Slider eingefügt
# Innerhalb der Graphen wurde eine Highlighter-Linie eingefügt
# Als Selecter wurde ein Slider eingestellt, mit dem die Jahreszahl ausgewählt werden kann

# 'Slider' zur Darstellung einzelner Jahre
year_slider = alt.binding_range(min=2000, max=2022, step=1)
slider_selection = alt.selection_single(bind=year_slider, fields=['year'], name='Jahr')


# Plot mit Preisen nach Waren aufgeteilt über den gesamten Zeitraum
line = alt.Chart(
    df
).mark_line(tooltip=True).encode(
    x=alt.X('date:T',
            axis=alt.Axis(title='Zeitraum')),
    y=alt.Y('mean_category:Q',
            axis=alt.Axis(title='Rohstoffpreis (€), Durchschnitt'),
            scale=alt.Scale(domain=[0,1800])),
    color=alt.Color('category:N',
                    scale=alt.Scale(domain=domain, range=range_),
                    sort=alt.EncodingSortField('mean_category', op='mean', order='descending'),
                    title='Rohstoffe')
).properties(title='Rohstoffpreise pro Jahr, kategorisiert',width=1600, height=800).transform_loess('date', 'mean_category',bandwidth=.005, groupby=['category']).mark_line(size=3)


# Highlighter zum gleichzeitigen Ablesen der y-Werte
highlight = alt.selection(type='single', 
                          on='mouseover', 
                          fields=['date'],
                          nearest=True,
                          )

selectors = alt.Chart(df).mark_point(tooltip=True, color='black').encode(
    x='date',
    opacity=alt.value(0)
).add_selection(highlight)

points = line.mark_point(tooltip=True, color='black').encode(
    opacity=alt.condition(highlight, alt.value(1), alt.value(0))
)

text = line.mark_text(align='left', dx=5, dy=-5, color='black').encode(
    text=alt.condition(highlight, 'mean_category:Q', alt.value(' '))
)

rules = alt.Chart(df).mark_rule(color='gray').encode(
    x='date',
).transform_filter(highlight)


# Verknüpfung der Plots und Slider zu Diagramm 2
fig_2 = alt.layer(line, selectors, points, rules, text).add_selection(slider_selection).transform_filter(slider_selection)

# Quelle Highlighter: https://altair-viz.github.io/gallery/multiline_tooltip.html
# Quelle Slider: https://altair-viz.github.io/gallery/multiple_interactions.html

In [7]:
# fig_2

Nehmen wir die Jahre unter die Lupe, lassen sich die ersten groben Beobachtungen noch einmal bestätigen. Alle Werte liegen im Jahr 2000 unter 400€. 2022 sind dort nurnoch Energie, Lebendvieh und Softs angesiedelt. Getreide erreicht derweil fast einen Wert von 700€, und Die Preise für beide Metallkategorien erreichen Preise von ca. 900-1100€. Lebendvieh stellt sich über die Jahre als stabilste Kategorie heraus, gefolgt von Softs. Lange Zeit gehört auch Energie zu den stabileren Kategorien. Mit Ausbruch des Ukraine-Krieges (25. Februar 2022) gerät aber auch dieser Durchschnittspreis ins Schwanken. Zurückführen lässt sich dies auf den drohenden Import-Stopp von Öl und Gas aus Russland. Es lässt sich im Detail sehen, dass Getreide und Industriemetalle deutlich den größten preislichen Schwankungen unterliegen. Sie verlaufen, nicht konsequent oder paralell, doch aber verlgeichbar ab 2008 bis 2022.<br>
<br>
Auch in diesem Plot lassen sich die ersten, groben Erkenntnisse des ersten Plots bestätigen und sogar verfeinern: Industrielle Metalle und Getreide unterliegen den größten preislichen Schwankungen, Lebendvieh den geringsten. 2007/08 und 2022 sind die auffälligsten Jahre für ersteren Kategorien. 